In [ ]:
from pymongo import MongoClient
import os
import pymongo
import pandas as pd
import requests

# Create a client
# client = MongoClient('mongodb://localhost:27017/')
# Read the connection string from the config.env file
with open('../server/config.env', 'r') as file:
    connection_string = file.read().strip()

connection_string = connection_string.replace('ATLAS_URI=', '')
# print(connection_string)

# Create a client
client = pymongo.MongoClient(connection_string)

In [ ]:
# Connect to your database
db = client['sample_analytics']

# Get your collection
collection = db['accounts']

# Retrieve all documents
documents = collection.find()

# Print the top 10 documents
for document in documents[:10]:
    print(document)

In [ ]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TOP_GAINERS_LOSERS&apikey=X4JSLVEG434TJWZX'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
# Convert the data to a dataframe
import pandas as pd

# Convert the data to a dataframe
df = pd.DataFrame.from_dict(data)
# print(df)

df.info()
df.describe()

In [ ]:
# Remove metadata and last_updated columns
df = df.drop(['metadata', 'last_updated'], axis=1)
df

In [ ]:
# Within the top_gainers column, there is a JSON formatted string that needs to be converted to a dataframe
df_top_gainers = pd.DataFrame.from_records(df['top_gainers'])
df_top_gainers

In [ ]:
def get_stock_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data['Time Series (Daily)']).T  # Transpose the data
    df.index = pd.to_datetime(df.index)  # Convert the index to datetime
    return df

# Example usage:
df = get_stock_data('MSFT', 'X4JSLVEG434TJWZX')
df

In [ ]:
# List of stock symbols
symbols = ['AAPL', 'GOOG', 'TSLA', 'MSFT']  # Replace with your list of symbols

# DataFrame to store all the data
all_data = pd.DataFrame()

for symbol in symbols:
    print(f"Getting data for {symbol}")
    df = get_stock_data(symbol, 'X4JSLVEG434TJWZX')
    df['Symbol'] = symbol  # Add a column for the stock symbol
    all_data = pd.concat([all_data, df])

# Now all_data contains the data for all the stocks

# Print the first 5 rows
all_data.head()

# Save the data to a csv file
# all_data.to_csv('stock_data.csv')

In [ ]:
import yfinance as yf

# List of stock symbols
symbols = ['D05.SI', 'U11.SI', 'O39.SI']  # Replace with your list of symbols


# DataFrame to store all the data
all_data = pd.DataFrame()

for symbol in symbols:
    print(f"Getting data for {symbol}")
    df = yf.download(symbol, start='2020-01-01', end='2022-12-31')
    df['Symbol'] = symbol  # Add a column for the stock symbol
    all_data = pd.concat([all_data, df])

# Now all_data contains the data for all the stocks


# Save the data to a CSV file
# all_data.to_csv('stock_data.csv')

In [ ]:
# Print the first 5 rows of each stock symbol
for symbol in symbols:
    print(f"Data for {symbol}")
    print(all_data[all_data['Symbol'] == symbol].head())


In [ ]:
# Read in the data from the txt file
path = "C:/Github/FYP/Data/Complete-List-of-Listed-Companies-on-Singapore-Stock-Exchange-052923.csv"

# Read the data into a DataFrame
SGXdf = pd.read_csv(path)


In [ ]:
SGXdf['Code']

# add .SI to the stock code
SGXdf['Code'] = SGXdf['Code'] + '.SI'

# Print the first 5 rows
SGXdf.head()

In [77]:
# Get the unique sectors
sectors = SGXdf['Sector'].unique()

# Create a dictionary to store the data for each sector
sector_data = {}

for sector in sectors:
    print(f"Processing sector {sector}")
    
    df_sector = SGXdf[SGXdf['Sector'] == sector]
    
    # Store the sector data in the dictionary
    sector_data[sector] = df_sector

# Now sector_data is a dictionary where the keys are sectors and the values are DataFrames with the data for each sector

Processing sector Real Estate
Processing sector Industrial & Commercial Services
Processing sector Technology Equipment
Processing sector Industrial Goods
Processing sector Food & Beverages
Processing sector Energy - Fossil Fuels
Processing sector Mineral Resources
Processing sector nan
Processing sector Cyclical Consumer Services
Processing sector Holding Companies
Processing sector Healthcare Services
Processing sector Software & IT Services
Processing sector Banking & Investment Services
Processing sector Chemicals
Processing sector Utilities
Processing sector Transportation
Processing sector Cyclical Consumer Products
Processing sector Retailers
Processing sector Telecommunications Services
Processing sector Automobiles & Auto Parts
Processing sector Personal & Household Products & Services
Processing sector Food & Drug Retailing
Processing sector Collective Investments
Processing sector Insurance
Processing sector Pharmaceuticals & Medical Research
Processing sector Financial Tech

In [78]:
# Print the first 5 rows of the 'Real Estate' sector
sector_data['Real Estate'].head()

,S.No.,Trading Name,Code,Sector
0,1,3Cnergy,502.SI,Real Estate
16,17,Adventus,5EF.SI,Real Estate
22,23,AIMS APAC Reit,O5RU.SI,Real Estate
23,24,AIMS Property,BVP.SI,Real Estate
28,29,Alset,40V.SI,Real Estate


In [86]:
# Get the unique sectors
sectors = SGXdf['Sector'].unique()
sectors

array(['Real Estate', 'Industrial & Commercial Services',
       'Technology Equipment', 'Industrial Goods', 'Food & Beverages',
       'Energy - Fossil Fuels', 'Mineral Resources', nan,
       'Cyclical Consumer Services', 'Holding Companies',
       'Healthcare Services', 'Software & IT Services',
       'Banking & Investment Services', 'Chemicals', 'Utilities',
       'Transportation', 'Cyclical Consumer Products', 'Retailers',
       'Telecommunications Services', 'Automobiles & Auto Parts',
       'Personal & Household Products & Services',
       'Food & Drug Retailing', 'Collective Investments', 'Insurance',
       'Pharmaceuticals & Medical Research',
       'Financial Technology (Fintech) & Infrastructure',
       'Applied Resources', 'Consumer Goods Conglomerates',
       'Academic & Educational Services', 'Renewable Energy'],
      dtype=object)

In [98]:
# Combine data for Real Estate and Industrial & Commercial Services sectors
RealAssetsSectors = ['Real Estate', 'Industrial & Commercial Services']
RealAssetsDF = SGXdf[SGXdf['Sector'].isin(RealAssetsSectors)]

# Verify that the data is correct
RealAssetsDF.head()

,S.No.,Trading Name,Code,Sector
0,1,3Cnergy,502.SI,Real Estate
1,2,5E Resources,NLC.SI,Industrial & Commercial Services
6,7,Abundance Intl,541.SI,Industrial & Commercial Services
9,10,Acesian Partners,5FW.SI,Industrial & Commercial Services
11,12,AcroMeta,43F.SI,Industrial & Commercial Services


In [ ]:
# call the yfinance API to get the stock data for the stock codes

# Get the stock codes from the 'Code' column of your DataFrame
symbols = SGXdf['Code'].tolist()

# DataFrame to store all the data
all_data = pd.DataFrame()

for symbol in symbols:
    print(f"Getting data for {symbol}")
    data = yf.download(symbol, start='2020-01-01', end='2022-12-31')
    data['Symbol'] = symbol  # Add a column for the stock symbol
    all_data = pd.concat([all_data, data])

# Now all_data contains the data for all the stocks

# Print the first 5 rows
print(all_data.head())

In [ ]:
# Print the first 5 rows of each stock symbol
for symbol in symbols:
    print(f"Data for {symbol}")
    print(all_data[all_data['Symbol'] == symbol].head())


In [ ]:
# Creating a moving average feature
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_10'] = df['Close'].rolling(window=10).mean()